<a href="https://colab.research.google.com/github/subineda01/sb/blob/main/trade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import plotly.express as px

In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
train_df = pd.read_csv("/content/drive/MyDrive/dacon/jeju/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/dacon/jeju/test.csv")
trade_df = pd.read_csv("/content/drive/MyDrive/dacon/jeju/international_trade.csv")

In [4]:
train_df['year'] = train_df['timestamp'].apply(lambda x : int(x[0:4]))
train_df['month'] = train_df['timestamp'].apply(lambda x : int(x[5:7]))
train_df['day'] = train_df['timestamp'].apply(lambda x : int(x[8:10]))

test_df['year'] = test_df['timestamp'].apply(lambda x : int(x[0:4]))
test_df['month'] = test_df['timestamp'].apply(lambda x : int(x[5:7]))
test_df['day'] = test_df['timestamp'].apply(lambda x : int(x[8:10]))

In [5]:
train_df["Y"]=np.nan # 새로운 열

In [6]:
X = np.random.normal(0,1,size = (1274,5))  # 랜덤한 하게 숫자 만듬 1274는 무역 데이터의 행의 길이고 5는 품목이 다섯개 잇으니까 각 품목마다 상관계수가 뽑힐거라고 생각해서 이캐 만듬

In [7]:
trade_df["price_trade"] = trade_df["무역수지"]/ (trade_df["수출 중량"] - trade_df["수입 중량"]) # 이건 일단 임의로 내가 대충 생각해서 조정한거임

In [8]:
corporations = ["TG", "BC", "RD", "CR", "CB"]

for i, name in enumerate(corporations):
    trade_df[name] = trade_df["price_trade"] * X[:, i] # 상관계수와 무역수지 조정값을 곱해서 trade_df에 저장해둠

In [9]:
trade_df['year'] = trade_df['기간'].apply(lambda x : int(x[0:4]))
trade_df['month'] = trade_df['기간'].apply(lambda x : int(x[5:7]))

In [10]:
trade_df.head()

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,price_trade,TG,BC,RD,CR,CB,year,month
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990,0.002776,0.000779,0.003264,-0.000792,0.000005,0.006345,2019,1
1,2019-01,양파,821330,222,4003206,1118,-896,0.000282,-0.000128,0.000324,0.000034,0.000032,0.000279,2019,1
2,2019-01,쪽파,60,1,93405,128,-127,0.001361,-0.000941,0.001340,-0.000437,-0.000840,0.000960,2019,1
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562,0.000880,-0.000629,-0.000495,-0.000685,0.000535,-0.001084,2019,1
4,2019-01,방울다다기 양배추,0,0,7580,38,-38,0.005013,0.003258,0.009177,0.002604,-0.006628,-0.005522,2019,1


In [11]:
corporations = ["TG", "BC", "RD", "CR", "CB"]
dfs = []  # 기업별 Y 데이터프레임을 저장할 리스트

for name in corporations:
    # 해당 기업의 평균 가격을 계산하여 데이터프레임 생성
    Y = trade_df.groupby(["year", "month"])[name].mean().reset_index()
    Y = Y.rename(columns={name: f"Y_{name}"})  # Y 열 이름 변경
    dfs.append(Y)  # 리스트에 추가

# 기업별 Y 데이터프레임을 병합
final_Y = dfs[0]
for i in range(1,len(dfs)):
    final_Y = final_Y.merge(dfs[i], on=["year", "month"], how="left")

# train_df와 병합
#train_df = train_df.merge(final_Y, on=["year", "month"], how="left")

In [12]:
final_Y[(final_Y["month"] == 3) & (final_Y["year"] == 2020)].iloc[:,4]

14   -0.000152
Name: Y_RD, dtype: float64

In [13]:
item_to_number = {
    "TG" : 2,
    "BC" : 3,
    "RD" : 4,
    "CR" : 5,
    "CB" : 6
}
for i in range(len(train_df)):
  months = train_df.iloc[i,8]
  years = train_df.iloc[i,7]
  items = train_df.iloc[i,2]
  j = item_to_number[items]
  if j == 2 or j == 4 :
    prev_month = months- 2
  else :
    prev_month = months -4


  if prev_month < 1:
    years = years-1
    if years == 2018:
      continue
    prev_month = prev_month + 12
  train_df.iloc[i,10] = final_Y[(final_Y["month"] == prev_month) & (final_Y["year"] == years)].iloc[:,j]

In [14]:
train_df.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,Y
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,NaN
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,NaN
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,NaN
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,NaN
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,NaN


In [15]:
duplicate_groups = train_df.groupby(['year', 'month', 'Y'])
duplicate_rows = duplicate_groups.filter(lambda x: len(x) > 1)
duplicate_rows # 23년 3월달을 제외하곤 전부 중복 행렬에 들어간 것으로 보아 잘 들어간것으로 보임

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,Y
59,TG_A_J_20190301,2019-03-01,TG,A,J,1700.0,4307.0,2019,3,1,-0.000169
60,TG_A_J_20190302,2019-03-02,TG,A,J,1967.0,4538.0,2019,3,2,-0.000169
61,TG_A_J_20190303,2019-03-03,TG,A,J,0.0,0.0,2019,3,3,-0.000169
62,TG_A_J_20190304,2019-03-04,TG,A,J,4891.0,3992.0,2019,3,4,-0.000169
63,TG_A_J_20190305,2019-03-05,TG,A,J,1210.0,4055.0,2019,3,5,-0.000169
...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,27,0.001220
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,28,0.001220
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,1,-0.001276
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,2,-0.001276


In [21]:
train_df.iloc[1,10]

nan

In [23]:
import math
month_list = [1]
count = 0
for i in range(len(train_df)):
  if math.isnan(train_df.iloc[i,10]):
    continue
  a = train_df.iloc[i,8]
  if a in month_list:
    train_df.iloc[i,10] = train_df.iloc[i,10] *(0.98 ** count)
    count +=1
  else:
    count = 0
    month_list.pop(0)
    month_list.append(a)
    train_df.iloc[i,10] = train_df.iloc[i,10] *(0.98 ** count)


In [24]:
item_list = list(train_df['item'].unique())
loc_list  = list(train_df['location'].unique())
corp_list = list(train_df['corporation'].unique())
for item in item_list:
  df_1 = train_df[train_df['item'] == item]
  for loc in loc_list:
    df_2 = df_1[df_1['location'] == loc]
    title = f"Graph of {item} in {loc}"
    fig = px.line(df_2, x='timestamp', y="Y", color='corporation', markers=True, title=title)
    fig.show()
